In [1]:
!pip3 install langchain openai pypdf tiktoken faiss-cpu jinja2


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


### Load your data


In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

openai_api_key = ""

In [3]:
loader = PyPDFLoader("../pdf/PDF.pdf")

In [4]:
data = loader.load()

In [5]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 2 document(s) in your data
There are 1734 characters in your document


### Chunk your data up into smaller documents


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

### Embedding

In [7]:
from langchain.chat_models import ChatOpenAI

# The vectorstore we'll be using
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)

In [8]:
# Get your embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to OpenAI
docsearch = FAISS.from_documents(docs, embeddings)

In [9]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# To parse outputs and get structured data back
from langchain.output_parsers import StructuredOutputParser, ResponseSchema



In [10]:
# The schema I want out
response_schemas = [
    ResponseSchema(name="name", description="O nome do candidato no currículo"),
    ResponseSchema(name="cellphone", description="O número de celular do candidato no currículo"),
    ResponseSchema(name="email", description="O email do candidato no currículo"),
    ResponseSchema(name="nationality", description="A nacionalidade do candidato no currículo"),
    ResponseSchema(name="state", description="A cidade e o estado de origem do candidato no currículo"),
    ResponseSchema(name="age", description="A idade do candidato no currículo"),
    ResponseSchema(name="professional_trajectory", description="Um array de dicionarios contendo o nome da empresa, o cargo, a data de início e de fim e um array de strings contendo os principais diferentes pontos de descrição do cargo"),
    ResponseSchema(name="academic_education", description="Um array de dicionarios contendo o nome da escola/universidade, o curso feito e a data de início e de fim"),
    ResponseSchema(name="courses", description="Um array de dicionarios contendo o nome do curso/certificado, o nome da empresa feito e o ano feito"),
    ResponseSchema(name="languages", description="Um array de dicionarios contendo o nome da língua e sua proficiência")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [11]:
# The format instructions that LangChain makes. Let's look at them
format_instructions = output_parser.get_format_instructions()

In [12]:
# The prompt template that brings it all together
# Note: This is a different prompt template than before because we are using a Chat Model
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("Dado o currículo de um candidato, extraia exatamente como escrito os dados a seguir \n \
                                                    {format_instructions} \n \
                                                    {context}")  
    ],
    partial_variables={"format_instructions": format_instructions},
    output_parser=output_parser
)

In [13]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                chain_type="stuff",
                                retriever=docsearch.as_retriever(),
                                chain_type_kwargs={"prompt": prompt},
                                verbose=True,
                            )

In [14]:
answer = qa.run('')
print(answer)



> Entering new RetrievalQA chain...

> Finished chain.
```json
{
	"name": "Felipe Takeo Kikuchi",
	"cellphone": "(11) 97465-3750",
	"email": "Kikuchi.takeo@hotmail.com",
	"nationality": "Brasileiro",
	"state": "Itatiba / SP",
	"age": "31 anos",
	"professional_trajectory": [
		{
			"nome_empresa": "Kromberg & Schubert do Brasil",
			"cargo": "Auxiliar de Produção",
			"data_inicio": "05/2010",
			"data_fim": "09/2010",
			"descricao": ["Montagem e acabamento final de chicotes elétricos automotivo."]
		},
		{
			"nome_empresa": "Kromberg & Schubert do Brasil",
			"cargo": "Inspetor de Qualidade",
			"data_inicio": "10/2010",
			"data_fim": "10/2017",
			"descricao": ["Monitoramento e acompanhamento de processos produtivos de montagem e de crimpagem, no que diz respeito à análise de modos de falha e ações corretivas.", "Verificação e liberação 100% em peças finais", "Leitura e interpretação de desenhos técnicos (produto e peça/componente)", "Gerenciamento, análise e tratativa de não con

In [15]:
obj = answer
output = output_parser.parse(obj)

### Informações Adicionais

#### Escrever aqui as informações não presentes no currículo

In [16]:
job_company = "RAZÃO SOCIAL DA VAGA"
job_title = "NOME DA POSIÇÃO"

by_name="NOME DO FEITO POR"
reviewed_by="NOME DA REVISÃO"

In [17]:
output['additional_information'] = """A carreira profissional do finalista Felipe foi construída na indústria automotiva, tendo atuado nos últimos (13) treze anos na Kromberg & Schubert, uma empresa fabricante de chicotes elétricos. Apesar de seu início na área produtiva, não demorou muito para que o candidato fosse oferecido a oportunidade de transferência para a área de qualidade, onde vem trabalhando desde então.

Durante sua trajetória, Felipe teve a oportunidade de atuar por um longo período na área de Inspeção,onde teve seu primeiro contato com fornecedores, participando de reuniões para buscar melhoriasnos processos em casos de não conformidade. No entanto, sua grande experiência com Qualidadede Fornecedores ocorreu nos últimos (05) cinco anos, quando o candidato assumiu o cargo de Analistade Qualidade. Desde então, Felipe vem trabalhando junto a um EQF em funções totalmente focadasna gestão de 57 fornecedores, sendo 48 destes manufatureiros (Injeção plástica, calhas, conectores,retentores, clipes, estamparia e cabos). Suas principais atividades estão na parte administrativa, sendo responsável pela elaboração dos PPAP, fornecendo suporte a todas as plantas da empresa na América Latina em casos de reclamações e oferecendo suporte à equipe de compras referente a certificações e análise de riscos dos fornecedores. Além disso, o candidato realiza visitas técnicas aos fornecedores para analisar os planos de ação e garantir melhorias, além de acompanhar o EQF (que atua como auditor líder) em auditorias VDA 6.3 e IATF 16949.

Após um longo período em sua empresa atual, Felipe está em busca de mudanças e novos desafios na área. Ao ser abordado com a oportunidade na Sumidenso, o finalista se atraiu muito pela possível transição da cultura alemã, presente em seu emprego atual, para a cultura japonesa, na qual ele tem grande interesse devido à sua descendência. O candidato possui um perfil organizado e metódico, no entanto, valoriza a colaboração e não demonstra dificuldades em manter boas relações mesmo em situações complexas. O foco em detalhes e estabilidade são destaques em seu perfil, assim como sua preferência por ambientes estruturados e rotinas bem estabelecidas. """



### Pacote de Remuneração

In [18]:
salary_claim = "6.800,00"

employment_relationship = "CLT"
gross_monthly_fixed_salary = "4.500,00"
plr = "2.300,00"
meal_voucher = "23,40"
food_voucher = "220,00"
medical_assistance = "Intermédica, sem coparticipação e extensiva para dependentes."
dental_care = "N/A"
life_insurance = "Itaú"
private_pension = "N/A"
other_benefits = "Veículo coorporativo (Polo 2022/2023), partilhado entre colaboradores e mediante agendamento prévio para períodos longos e curtos."

#### Inputs sobre os idiomas presentes no currículo

Abrirá uma caixa de texto onde deverá ser colocado as descrições dos 6 tipos de avaliações.
eg:


*PRONÚNCIA*: É influenciada pela língua materna, mas apenas algumas vezes interfere na compreensão.

*ESTRUTURA*: Aspectos gramaticais e padrões estruturais das frases são geralmente bem controlados. Erros
podem acontecer em circunstâncias inesperadas.

*VOCABULÁRIO*: A precisão do vocabulário é geralmente suficiente para comunicar-se em tópicos co-
muns e relacionados ao trabalho.

*FLUÊNCIA*: Capaz de se expressar verbalmente e se fazer compreendido, mas a construção das fra-
ses e pausas são geralmente inapropriadas, mas não comprometem a compreensão do discurso.

*COMPREENSÃO*: Na maioria das vezes, é precisa em tópicos comuns e relacionados ao trabalho quando
o sotaque é suficientemente compreensível para uma comunidade internacional.

*INTERAÇÕES*: As respostas costumam ser imediatas, apropriadas e informativas, e consegue iniciar e
manter diálogos mesmo quando surgem eventos inesperados.

In [20]:
linguas = []

for idioma_info in output["languages"]:
    idioma = idioma_info["nome_lingua"]
    proficiencia = idioma_info["proficiencia"]

    # Pergunta ao usuário o nível (level) do idioma
    level = int(input(f"Nível de {idioma} de 1 à 6: "))

    # Inicializa o dicionário para armazenar os resultados
    resultado = {"idioma": idioma, "level": level}

    # Se o level for maior que 1, coleta as informações adicionais
    if level > 1:
        resultado["pronuncia"] = input("Avalie a pronúncia: ")
        resultado["estrutura"] = input("Avalie a estrutura: ")
        resultado["vocabulario"] = input("Avalie o vocabulário: ")
        resultado["fluencia"] = input("Avalie a fluência: ")
        resultado["compreensao"] = input("Avalie a compreensão: ")
        resultado["interacoes"] = input("Avalie as interações: ")
    else:
        resultado[
            "pre_elementar"
        ] = "Não possui conhecimento do idioma: não se sentiu confortável em fazer a avaliação oral com a Beyond HR."

    # Adiciona o resultado à lista de resultados
    linguas.append(resultado)

### Python para Latex

In [21]:
def format_latex_personal_info(data):
    formatted_string = (
        r"""
\begin{enumerate}[label={}, leftmargin=0pt, topsep=8pt, itemsep=0pt]
  \item """
        + data.get("state", "")
        + r"""
  \item """
        + data.get("cellphone", "")
        + r"""
  \item """
        + data.get("email", "")
        + r"""
  \item """
        + data.get("nationality", "")
        + ", "
        + data.get("age", "")
        + r"""
\end{enumerate}
"""
    )
    return formatted_string


def format_latex_name(data):
    template = (
        r"""\begin{minipage}[t]{0.4\textwidth}
  """
        + data.get("job_position", "")
        + r""" \\
  """
        + data.get("name", "")
        + r"""
\end{minipage}
"""
    )
    return template


def format_latex_professional_trajectory(data):
    sorted_trajectory = sorted(
        data["professional_trajectory"], key=lambda x: x["data_fim"], reverse=True
    )

    grouped_data = {}
    for item in sorted_trajectory:
        empresa = item["nome_empresa"]
        if empresa not in grouped_data:
            grouped_data[empresa] = []
        grouped_data[empresa].append(item)

    latex_strings = {}
    for empresa, cargo_data in grouped_data.items():
        empresa = empresa.replace("&", r"\&").replace("%", r"\%")
        table = (
            r"""{\renewcommand{\arraystretch}{1.6}
\begin{tabular}{@{}p{0.68\textwidth}p{0.3\textwidth}@{}}
\textbf{"""
            + empresa
            + r"""} & """
            + cargo_data[0]["data_inicio"]
            + " - "
            + cargo_data[0]["data_fim"]
            + r""" \\
            """
        )

        for item in cargo_data:
            cargo = item["cargo"].replace("&", r"\&").replace("%", r"\%")
            row = (
                r""""""
                + cargo
                + r""" & """
                + item["data_inicio"]
                + " - "
                + item["data_fim"]
                + r""" \\
        """
            )
            table += row

        table += r"""\end{tabular}
        }"""
        latex_strings[empresa] = table

    lista_de_strings = list(latex_strings.values())

    string_completa = "".join(lista_de_strings)

    return string_completa


def format_latex_professional_experience(data):
    sorted_trajectory = sorted(
        data["professional_trajectory"], key=lambda x: x["data_fim"], reverse=True
    )

    latex_code = ""
    seen_empresas = set()

    for entry in sorted_trajectory:
        empresa = entry["nome_empresa"].upper().replace("&", r"\&").replace("%", r"\%")
        if empresa not in seen_empresas:
            latex_code += f"\\vspace{{10pt}}\n"
            latex_code += f"\\section{{{empresa}}}\n\n"
            seen_empresas.add(empresa)

        cargo = entry["cargo"].replace("&", r"\&").replace("%", r"\%")
        data_inicio = entry["data_inicio"]
        data_fim = entry["data_fim"]
        descricao = entry["descricao"]

        latex_code += f"{cargo}\n"
        latex_code += f"\\vspace{{3pt}}\\\\\n"
        latex_code += f"{{\\footnotesize {data_inicio} – {data_fim}}}\n"

        if descricao:
            latex_code += "\\begin{enumerate}[leftmargin=32pt, topsep=15pt, itemsep=8pt, labelsep=10pt, align=left]\n"
            for item in descricao:
                latex_code += f"  \\item {item}\n".replace("&", r"\&").replace(
                    "%", r"\%"
                )
            latex_code += "\\end{enumerate}\n\n"

    return latex_code


def format_latex_academic(data):
    sorted_education = sorted(
        data["academic_education"], key=lambda x: x["data_inicio"], reverse=True
    )
    latex_code = "\\begin{enumerate}[label={\\resizebox{0.4em}{!}{$\\blacksquare$}}, leftmargin=32pt, topsep=15pt, itemsep=8pt, labelsep=10pt, align=left]"

    for item in sorted_education:
        nome_escola = item["nome_escola"]
        curso = item["curso"]
        data_inicio = item["data_inicio"]

        latex_code += f"\\item {curso}\n"
        latex_code += f"\\newline {nome_escola} – {data_inicio}\n"

    latex_code += "\\end{enumerate}"
    return latex_code


def format_latex_courses(data):
    sorted_courses = sorted(data["courses"], key=lambda x: x["ano_feito"], reverse=True)

    latex_code = "\\begin{enumerate}[label={\\resizebox{0.4em}{!}{$\\blacksquare$}}, leftmargin=32pt, topsep=15pt, itemsep=0pt, labelsep=10pt, align=left]\n"

    for course in sorted_courses:
        nome_curso = course["nome_curso"]
        nome_empresa = course["nome_empresa"]
        ano_feito = course["ano_feito"]

        if nome_empresa:
            item = f"{nome_curso} – {nome_empresa} – {ano_feito}"
        else:
            item = f"{nome_curso} – {ano_feito}"

        latex_code += f"  \\item {item}\n"

    latex_code += "\\end{enumerate}"

    return latex_code


def format_latex_languages(data):
    latex_code = ""
    levels = [
        "Pré-Elementar",
        "Elementar",
        "Pré-Operacional",
        "Operacional",
        "Avançado",
        "Fluente",
    ]

    for idioma_info in data:
        idioma = idioma_info["idioma"]
        level = idioma_info["level"]

        # Inicializa o código LaTeX para o idioma
        latex_code += f"\\vspace{{8pt}}\n"
        latex_code += f"\\textbf{{{idioma.upper()} ({level}): "

        # Gera a lista de níveis de proficiência
        latex_code += f"{levels[level-1]}".upper()
        latex_code += "}"
        latex_code += "\n\\begin{enumerate}[leftmargin=32pt, topsep=16pt, itemsep=-2pt, align=left]\n"

        # Adiciona os níveis de proficiência de 1 a 6
        for i in range(1, 7):
            # Usa uma formatação condicional para aplicar negrito
            if i == level:
                latex_code += f"  \\item \\textbf{{{levels[i-1]}}}\n"
            else:
                latex_code += f"  \\item {levels[i-1]}\n"

        latex_code += "\\end{enumerate}\n"
        latex_code += "\\vspace{15pt}\n"

        if level == 1:
            latex_code += f"\\textbf{{Pré-Elementar}}: {idioma_info['pre_elementar']}\n"
        else:
            latex_code += f"\\textbf{{PRONÚNCIA}}: {idioma_info['pronuncia']}\n"
            latex_code += "\\newline\\newline\n"
            latex_code += f"\\textbf{{ESTRUTURA}}: {idioma_info['estrutura']}\n"
            latex_code += "\\newline\\newline\n"
            latex_code += f"\\textbf{{VOCABULÁRIO}}: {idioma_info['vocabulario']}\n"
            latex_code += "\\newline\\newline\n"
            latex_code += f"\\textbf{{FLUÊNCIA}}: {idioma_info['fluencia']}\n"
            latex_code += "\\newline\\newline\n"
            latex_code += f"\\textbf{{COMPREENSÃO}}: {idioma_info['compreensao']}\n"
            latex_code += "\\newline\\newline\n"
            latex_code += f"\\textbf{{INTERAÇÕES}}: {idioma_info['interacoes']}\n"
        latex_code += "\\vspace{25pt}\n"
        latex_code += "\\newline\\newline\n"

    return latex_code

In [22]:
personal_info = format_latex_personal_info(
    {
        "state": output["state"],
        "cellphone": output["cellphone"],
        "email": output["email"],
        "nationality": output["nationality"],
        "age": output["age"],
    }
)

name_job = format_latex_name(
    {
        "name": output["name"].upper(),
        "job_position": job_title.upper(),
    }
)

professional_trajectory = format_latex_professional_trajectory(
    {
        "professional_trajectory": output["professional_trajectory"],
    }
)

professional_experience = format_latex_professional_experience(
    {
        "professional_trajectory": output["professional_trajectory"],
    }
)

academic_education = format_latex_academic(
    {
        "academic_education": output["academic_education"],
    }
)


courses = format_latex_courses(
    {
        "courses": output["courses"],
    }
)

languages = format_latex_languages(linguas)


print(
    personal_info,
    name_job,
    professional_trajectory,
    professional_experience,
    courses,
    languages,
)


\begin{enumerate}[label={}, leftmargin=0pt, topsep=8pt, itemsep=0pt]
  \item Itatiba / SP
  \item (11) 97465-3750
  \item Kikuchi.takeo@hotmail.com
  \item Brasileiro, 31 anos
\end{enumerate}
 \begin{minipage}[t]{0.4\textwidth}
  NOME DA POSIÇÃO \\
  FELIPE TAKEO KIKUCHI
\end{minipage}
 {\renewcommand{\arraystretch}{1.6}
\begin{tabular}{@{}p{0.68\textwidth}p{0.3\textwidth}@{}}
\textbf{Kromberg \& Schubert do Brasil} & 12/2017 - até o momento \\
            Analista de Qualidade Jr / Pl. & 12/2017 - até o momento \\
        Inspetor de Qualidade & 10/2010 - 10/2017 \\
        Auxiliar de Produção & 05/2010 - 09/2010 \\
        \end{tabular}
        } \vspace{10pt}
\section{KROMBERG \& SCHUBERT DO BRASIL}

Analista de Qualidade Jr / Pl.
\vspace{3pt}\\
{\footnotesize 12/2017 – até o momento}
\begin{enumerate}[leftmargin=32pt, topsep=15pt, itemsep=8pt, labelsep=10pt, align=left]
  \item Gerenciamento de atividades da Gestão de Qualidade de Fornecedores para todas as plantas da América Lat

In [23]:
import jinja2
import subprocess
import os

# Diretório onde o script Python está localizado
notebook_directory = os.getcwd()

# Dicionário de variáveis
dados = {
    "name": name_job,
    "personalInfo": personal_info,
    "professionalTrajectory": professional_trajectory,
    "professionalExperience": professional_experience,
    "academicEducation": academic_education,
    "courses": courses,
    "languages": languages,
    "aditionalInformation": output["additional_information"]
    .replace("\n", r"\\")
    .replace("&", r"\&")
    .replace("%", r"\%"),
    "salaryClaim": salary_claim,
    "employmentRelationship": employment_relationship,
    "grossMonthlyFixedSalary": gross_monthly_fixed_salary,
    "plr": plr,
    "mealVoucher": meal_voucher,
    "foodVoucher": food_voucher,
    "medicalAssistance": medical_assistance,
    "dentalCare": dental_care,
    "lifeInsurance": life_insurance,
    "privatePension": private_pension,
    "otherBenefits": other_benefits,
    "jobCompany": "\\section{"
    + job_company
    + "}".replace("&", r"\&").replace("%", r"\%"),
    "author": by_name,
    "reviewer": reviewed_by,
}

# Caminho para o template LaTeX
template_path = os.path.join(notebook_directory, "report_template.tex")

# Carrega o template LaTeX
template_loader = jinja2.FileSystemLoader(searchpath=os.path.dirname(template_path))
template_env = jinja2.Environment(loader=template_loader)
template = template_env.get_template(os.path.basename(template_path))

# Preenche o template com os dados
latex_content = template.render(dados)

# Caminho para o arquivo temp.tex (no mesmo diretório do template)
temp_tex_path = os.path.join(os.path.dirname(template_path), "temp.tex")

# Salva o conteúdo LaTeX em um arquivo temporário
with open(temp_tex_path, "w") as temp_file:
    temp_file.write(latex_content)

# Compila o arquivo LaTeX para PDF
subprocess.run(["pdflatex", temp_tex_path])
subprocess.run(["pdflatex", temp_tex_path])

# Renomeia o arquivo PDF resultante
subprocess.run(["mv", "temp.pdf", "output.pdf"])

# Limpa arquivos temporários (opcional)
subprocess.run(["rm", "temp.aux", "temp.log"])

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(/Users/franciscogaieski/Documents/Beyond/byndhr-cv-report/latex/temp.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-02-22>
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/report.cls
Document Class: report 2022/07/02 v1.4n Standard LaTeX document class
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2023/texmf-dist/tex/latex/tex-gyre/tgadventor.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/kvoptions/kvoptions.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2023/texmf-dist/tex/generic/ltxcmds/ltxcmds.sty)
(/usr/local/texlive/2023/texmf-dist/tex/latex/kvsetkeys/kvsetkeys.sty)))
(/usr/local/texlive/2023/texmf-dist/tex/latex/base/fontenc.sty
(/usr/local/texlive/2023/texmf-dist/tex/latex/tex-gyre/t1qag.fd))
(/usr/local/texlive/2023/texmf-dist/te

CompletedProcess(args=['rm', 'temp.aux', 'temp.log'], returncode=0)